# LACSS GRPC call Demo

This is a small notebook demonstrating LACSS model inference by GRPC call to an inference server




## Setting up the environment

In [ ]:
!pip install git+https://github.com/jiyuuchc/lacss
!pip install --upgrade protobuf

In [ ]:
import cv2
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import numpy as np
import grpc

import lacss.deploy.proto as proto
from lacss.utils import show_images

SERVER = "lacss.cam.uchc.edu"

## Download some data for testing

In [ ]:
!wget -c https://data.mendeley.com/public-files/datasets/894mmsd9nj/files/568e524f-9a95-45a6-9f80-3619969c2a37/file_downloaded -O images.zip

import zipfile

data_path = 'image_data'
with zipfile.ZipFile('images.zip', "r") as f:
    f.extractall(data_path)

image = imageio.imread("image_data/test/000_img.png")

plt.imshow(image)

## Call GRPC server

In [ ]:
# construct request
pixels = proto.Pixels(
    bindata = proto.BinData(data=image.tobytes()),
    size_x = image.shape[1],
    size_y = image.shape[0],
    size_c = image.shape[2],
    dimension_order = "CXYZT",
    dtype = "u1", # uint8
)

settings = proto.DetectionSettings(
    scaling_hint = 1.0,
)

request = proto.DetectionRequest(
    image_data = proto.ImageData(pixels=pixels),
    detection_settings = settings,
)

# call server
with grpc.secure_channel(target=SERVER, credentials=grpc.ssl_channel_credentials()) as channel:
    stub = proto.LacssStub(channel)
    response = stub.RunDetection(request)

# generate label
label = np.zeros(image.shape[:2], dtype="uint8")

for k, det in enumerate(response.detections):

    polygon = [[p.x, p.y] for p in det.roi.polygon.points]
    polygon = np.round(np.array(polygon)).astype(int)

    cv2.fillPoly(label, [polygon], k + 1)

show_images([image, label])